In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import zipfile
import os

# Define paths
zip_path = "/content/drive/MyDrive/images.zip"  # Update with correct file name
extract_path = "/content/images"  # Destination folder

# Create extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction Complete! Files are in:", extract_path)


Extraction Complete! Files are in: /content/images


In [4]:
!pip install transformers torchvision torch pandas pillow



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForImageClassification
import torch
from torchvision import transforms
from PIL import Image
from PIL import Image, UnidentifiedImageError
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score
import pandas as pd
from tqdm import tqdm
import zipfile
import os

In [5]:
df=pd.read_csv("/content/labels.csv")
image_folder="/content/images/images"
df["image_name"]= df["image_name"].apply(lambda x: os.path.join(image_folder, x))
Image.open(df["image_name"][500])
for i in range(len(df["overall_sentiment"])):
    if(i=="very_positive" or i=="positive"):
        df["overall_sentiment"]=2
    if(i=="very_negative" or i=="negative"):
        df["overall_sentiment"]=0
    else:
        df["overall_sentiment"]=1
print(df["overall_sentiment"][221])
df = df.reset_index(drop=True)

1


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:

class MultimodalDataset(Dataset):
    def __init__(self, text_data, image_data, labels, text_tokenizer, image_transform, max_length=128):
        self.text_data = text_data
        self.image_data = image_data
        self.labels = labels
        self.text_tokenizer = text_tokenizer
        self.image_transform = image_transform
        self.max_length = max_length

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        text_input = self.text_data[idx]

        if isinstance(text_input, float):
            processed_text_input = str(text_input)
        elif isinstance(text_input, str):
            processed_text_input = text_input
        elif isinstance(text_input, list):
            processed_text_input = " ".join(text_input)
        else:
            raise ValueError(f"Unexpected text input type: {type(text_input)}")

        try:
            image_input = Image.open(self.image_data[idx]).convert("RGB")
        except (UnidentifiedImageError, OSError) as e:
            print(f"Skipping image at index {idx} due to error: {e}")
            image_input = Image.new("RGB", (224, 224), color=(255, 255, 255))  # white placeholder image

        label = self.labels[idx]
        text_inputs = self.text_tokenizer(
            processed_text_input,
            return_tensors="pt",
            padding='max_length',
            truncation=True,
            max_length=self.max_length
        )

        input_ids = text_inputs["input_ids"].squeeze(0)
        attention_mask = text_inputs["attention_mask"].squeeze(0)

        image_tensor = self.image_transform(image_input)

        return input_ids, attention_mask, image_tensor, label



text_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = MultimodalDataset(df["text_corrected"], df["image_name"], df["overall_sentiment"], text_tokenizer, image_transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

text_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3).to(device)
image_model = AutoModelForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=3).to(device)

final_classifier = nn.Linear(6, 3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(
    list(text_model.parameters()) + list(image_model.parameters()) + list(final_classifier.parameters()),
    lr=1e-5
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weig

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import torch

num_epochs = 5
for epoch in range(num_epochs):
    text_model.train()
    image_model.train()
    final_classifier.train()

    running_loss = 0.0
    all_preds = []
    all_labels = []

    with tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch") as pbar:
        for batch_idx, (input_ids, attention_mask, image_tensor, label) in enumerate(pbar):
            # Move data to CUDA device
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            image_tensor = image_tensor.to(device)
            label = label.to(device)

            # Forward pass for text and image models
            text_inputs = {"input_ids": input_ids, "attention_mask": attention_mask}
            text_outputs = text_model(**text_inputs)
            text_cls_output = text_outputs.logits

            image_outputs = image_model(image_tensor)
            image_cls_output = image_outputs.logits

            # Combine text and image outputs
            combined_output = torch.cat((text_cls_output, image_cls_output), dim=1)
            final_output = final_classifier(combined_output)

            # Compute loss and gradients
            loss = criterion(final_output, label)
            running_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track predictions and labels
            _, preds = torch.max(final_output, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(label.cpu().numpy())

            pbar.set_postfix(loss=running_loss / (batch_idx + 1))

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=1)
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {running_loss / len(train_loader):.4f}")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")


Epoch 1/5:  70%|██████▉   | 153/219 [04:47<02:03,  1.87s/batch, loss=0.43]

Skipping image at index 5118 due to error: image file is truncated


Epoch 1/5:  79%|███████▉  | 174/219 [05:26<01:23,  1.86s/batch, loss=0.396]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 1/5: 100%|██████████| 219/219 [06:48<00:00,  1.87s/batch, loss=0.341]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/5 - Loss: 0.3412
Accuracy: 0.9245, Precision: 0.3333, Recall: 0.3082, F1-score: 0.3203


Epoch 2/5:   1%|▏         | 3/219 [00:05<06:41,  1.86s/batch, loss=0.116]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 2/5:  35%|███▌      | 77/219 [02:22<04:25,  1.87s/batch, loss=0.102]

...

Epoch 4/5 - Loss: 0.0317
Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1-score: 1.0000


Epoch 5/5:  33%|███▎      | 73/219 [02:14<04:23,  1.81s/batch, loss=0.0245]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 5/5:  96%|█████████▋| 211/219 [06:29<00:14,  1.83s/batch, loss=0.0221]

Skipping image at index 5118 due to error: image file is truncated


Epoch 5/5: 100%|██████████| 219/219 [06:43<00:00,  1.84s/batch, loss=0.0219]

Epoch 5/5 - Loss: 0.0219
Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1-score: 1.0000
